# import statements

In [1]:
import torch.nn.functional as F
import torch
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss
import pandas as pd

from torch.nn import LogSoftmax, NLLLoss,CrossEntropyLoss
#from scipy.optimize import fmin


# utils

In [2]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    m =  torch.max(x,dim=1,keepdims=True) 
    #print(m)
    e_x = torch.exp(torch.sub(x,m.values)) 
    return e_x / torch.sum(e_x,dim=1,keepdims=True)

In [3]:
def focus_(a,x):
  """
  focus function parametrized by a*x
  returns : averaged input for classification function
  """
  #print(a*x)
  batch = x.shape[0]
  m = x.shape[1]
  d = x.shape[2]
  out = torch.matmul(x,a)[:,:,0]
  #print(out.shape)
  out = softmax(out)
  #print(out,x,out*x)
  #print(out.shape)
  out = out.view(batch,m,1)*x

  #print(out)
  #out = torch.sum(out*x,dim=1)
  dat = torch.zeros((batch,d))
  for i in range(batch):
    dat[i] = torch.sum(out[i],dim=0)

  return dat

def classification_(b,c,x):
  """
  classification function parametrized by b*x + c
  returns  : softmax(b*x+c)
  """
  out = torch.matmul(x,b)

  #c = torch.tensor(np.broadcast_to(c.numpy(), out.shape) )

  out = torch.add(out ,c)
  out = out
  return out

In [4]:
def derv_g(b,c):
  """
   derivate of log-loss with respect to b and c using autograd
  """
  #print(yhat,y,yhat-y)
  #print()
  db = b.grad
  dc = c.grad
  # db = np.dot(xhat,yhat-y)/xhat.shape[0]
  # dc  = np.sum(yhat-y)/xhat.shape[0]
  return db,dc

In [5]:
def derv_f(a):
  """
  derivative of log-loss with respect to a using autograd

  """
  da = a.grad
  # da = np.sum((yhat-y)*b*(  ( (x[:,0] - x[:,1])* x[:,0] ) +  ( (x[:,1]- x[:,0]) * x[:,1] )   ) * (np.exp((a*x[:,0]+a*x[:,1]))/ (np.exp(a*x[:,0])+ np.exp(a*x[:,1]))**2 )) / xhat.shape[0] 
  
  # #print(  (  ( (x[:,0] - x[:,1])* x[:,0] ) +  ( (x[:,1]- x[:,0]) * x[:,1] )   ) * (np.exp((a*x[:,0]+a*x[:,1]))/ (np.exp(a*x[:,0])+ np.exp(a*x[:,1]))**2 ) )
    
  return da

In [6]:

def gd(w,dw):
  """
  updates given parameter in negative direction of gradient
  """
  #print(w,dw)
  eta = torch.tensor([0.01])
  with torch.no_grad():
    w = w - torch.mul(eta,dw)
  return w

# m = 2

In [7]:
# X = torch.tensor([[3,-1],[-1,3],[1,3],[3,1]]) # mosaic data m = 2 , d= 1
# Y = torch.tensor([0,0,1,1])


# a = torch.tensor([[0.],[0.]])
# b = torch.tensor([[0.,0.],[0.,0.]])
# c = torch.tensor([0.,0.])



X = torch.tensor([[[1,1],[3.5,4]],   [[3.5,4],[1,1]],   [[2,1],[3.5,4]], [[3.5,4],[2,1]], [[1,2],[3.5,4]], [[3.5,4],[1,2]]  ])
Y = torch.tensor([0,0,1,1,2,2])

In [8]:
def minimize_b_c(x,y,a,b,c,epochs=1000):
  # b = 0 
  # c = 0 

  #a.requires_grad=False
  criterion = CrossEntropyLoss()
  #y = y.float()

  with torch.no_grad():
    x_average = focus_(a,x)
    yhat = classification_(b,c,x_average)
    initial_loss = criterion(yhat,y)

  
  print("x average at 0 epoch", x_average )
  print("yhat at 0 epoch",yhat)
  print("loss at 0 epoch",criterion(yhat,y).item())

  if initial_loss <= 0.01:
    return b,c,initial_loss.item()


  for i in range(epochs):
    a.requires_grad = False
    b.requires_grad = True
    c.requires_grad = True
    x_average = focus_(a,x)
    yhat = classification_(b,c,x_average)

    loss = criterion(yhat,y)
    b.retain_grad()
    c.retain_grad()

    loss.backward()

    #der_b,der_c = derv_g(b,c)
    #print(i,der_b,der_c)
    grad_b = b.grad
    grad_c = c.grad
    b = gd(b,grad_b)
    c = gd(c,grad_c)
    
    
    x_average = focus_(a,x)
    yhat = classification_(b,c,x_average)
    current_loss = criterion(yhat,y) 
    #print(current_loss<=(initial_loss/2) , current_loss,initial_loss)
    if current_loss<= 0.01:
      break
  print("   ")
  with torch.no_grad():
    x_average = focus_(a,x)
    print("x average",x_average)
    yhat = classification_(b,c,x_average) 
    print("Y hat",yhat)
    current_loss  = criterion(yhat,y)

  print("Loss",current_loss.item(),i)
  
  return b,c,current_loss.item()



In [9]:
# a = torch.tensor([0.],requires_grad=True)
# b = torch.tensor([0.],requires_grad=True)
# c = torch.tensor([0.],requires_grad=True)


a = torch.tensor([[0.],[0.]],requires_grad=True)
b = torch.tensor([[0.,0.,0.],[0.,0.,0.]],requires_grad=True)
c = torch.tensor([0.,0.,0.],requires_grad=True)



b,c,loss = minimize_b_c(X,Y,a,b,c,200000)

x average at 0 epoch tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000],
        [2.2500, 3.0000]])
yhat at 0 epoch tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
loss at 0 epoch 1.0986123085021973
   
x average tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000],
        [2.2500, 3.0000]])
Y hat tensor([[ 2.4705, -1.3075, -1.1697],
        [ 2.4705, -1.3075, -1.1697],
        [ 1.8867,  5.7819, -7.6774],
        [ 1.8867,  5.7819, -7.6774],
        [ 1.6920, -7.1543,  5.4560],
        [ 1.6920, -7.1543,  5.4560]])
Loss 0.030338361859321594 199999


In [10]:
def minimize_a(x,y,a,b,c,epochs=1000):

  #b = 0 
  #c = 0 
  criterion =CrossEntropyLoss()
  #y = y.float()
  
  with torch.no_grad():
    x_average = focus_(a,x)
    yhat = classification_(b,c,x_average)
    initial_loss = criterion(yhat,y)

  if initial_loss <= 0.01:
    return a,initial_loss.item()
  print("x average at 0 epoch", x_average )
  print("yhat at 0 epoch",yhat)
  print("loss at 0 epoch",criterion(yhat,y).item())

  for i in range(epochs):
    a.requires_grad =True
    b.requires_grad = False
    c.requires_grad = False
    x_average = focus_(a,x)
    yhat = classification_(b,c,x_average)

    
    loss = criterion(yhat,y)
    a.retain_grad()
    loss.backward()

    der_a = derv_f(a)
    #print(i,der_a)
    a = gd(a,der_a)

    x_average = focus_(a,x)
    yhat = classification_(b,c,x_average)
    current_loss = criterion(yhat,y)
    if current_loss <= 0.01:
      break 
  print("*"*60)
  with torch.no_grad():
    x_average = focus_(a,x)
    print("x average",x_average)
    yhat = classification_(b,c,x_average) 
    print("Y hat",yhat)
    current_loss = criterion(yhat,y)
  print("Loss",current_loss.item(),i)
  return a,current_loss.item()

In [11]:
# a = torch.tensor([0.],requires_grad=True)
# b = torch.tensor([0.],requires_grad=False)
# c = torch.tensor([0.],requires_grad=False)


a = torch.tensor([[0.],[0.]],requires_grad=True)
b = torch.tensor([[0.,0.,0.],[0.,0.,0.]],requires_grad=True)
c = torch.tensor([0.,0.,0.],requires_grad=True)


a,loss = minimize_a(X,Y,a,b,c,200000)

x average at 0 epoch tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000],
        [2.2500, 3.0000]])
yhat at 0 epoch tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
loss at 0 epoch 1.0986123085021973
************************************************************
x average tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000],
        [2.2500, 3.0000]])
Y hat tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
Loss 1.0986123085021973 199999


# Alternate minimization

In [12]:
# a = torch.tensor([0.],requires_grad=True)
# b = torch.tensor([0.],requires_grad=True)
# c = torch.tensor([0.],requires_grad=True)


a = torch.tensor([[0.],[0.]],requires_grad=True)
b = torch.tensor([[0.,0.,0.],[0.,0.,0.]],requires_grad=True)
c = torch.tensor([0.,0.,0.],requires_grad=True)

criterion = CrossEntropyLoss()
data = pd.DataFrame(columns=["sno","b_c_fixed","a_fixed","a_value","b_value","c_value","loss"]) 

#Y_ = Y.float()

X_average = focus_(a,X)
Yhat = classification_(b,c,X_average)
initial_loss = criterion(Yhat,Y)

#print(initial_loss)

k = 0 
data.loc[k] = [k,True,True,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),initial_loss.item()]
k = k+1
j= 1

for i in range(0,40,2):
  print("Minimize b and c")
  b,c,loss = minimize_b_c(X,Y,a,b,c,200000)
  #print(b,c)
  data.loc[k] = [j,False,True,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),loss]
  print("*"*60)
  print("  ")
  print("minimize a")
  

  #print(a,b,c)

  a,loss = minimize_a(X,Y,a,b,c,200000)
  data.loc[k+1] = [j,True,False,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),loss]
  print("  ")
  k = k+2
  j = j+1

Minimize b and c
x average at 0 epoch tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000],
        [2.2500, 3.0000]])
yhat at 0 epoch tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
loss at 0 epoch 1.0986123085021973
   
x average tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000],
        [2.2500, 3.0000]])
Y hat tensor([[ 2.4705, -1.3075, -1.1697],
        [ 2.4705, -1.3075, -1.1697],
        [ 1.8867,  5.7819, -7.6774],
        [ 1.8867,  5.7819, -7.6774],
        [ 1.6920, -7.1543,  5.4560],
        [ 1.6920, -7.1543,  5.4560]])
Loss 0.030338361859321594 199999
************************************************************
  
minimize a
x average at 0 epoch tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.750

In [13]:
data

,sno,b_c_fixed,a_fixed,a_value,b_value,c_value,loss
0,0,True,True,"[[0.0], [0.0]]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[0.0, 0.0, 0.0]",1.098612
1,1,False,True,"[[0.0], [0.0]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.030338
2,1,True,False,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
3,2,False,True,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
4,2,True,False,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
5,3,False,True,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
6,3,True,False,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
7,4,False,True,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
8,4,True,False,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
9,5,False,True,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999


In [14]:
# data.to_csv("data_1_m_2_1.csv",index=False)

In [15]:
# a = torch.tensor([0.],requires_grad=True)
# b = torch.tensor([0.],requires_grad=True)
# c = torch.tensor([0.],requires_grad=True)


a = torch.tensor([[0.],[0.]],requires_grad=True)
b = torch.tensor([[0.,0.,0.],[0.,0.,0.]],requires_grad=True)
c = torch.tensor([0.,0.,0.],requires_grad=True)

criterion = CrossEntropyLoss()

data_1 = pd.DataFrame(columns=["sno","b_c_fixed","a_fixed","a_value","b_value","c_value","loss"]) 

X_average = focus_(a,X)
Yhat = classification_(b,c,X_average)
#Y_ = Y.float()
initial_loss = criterion(Yhat,Y)

k = 0 
data_1.loc[k] = [k,True,True,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),initial_loss.item()]
k = k+1
j= 1

for i in range(0,40,2):


  print("minimize a")
  a,loss = minimize_a(X,Y,a,b,c,200000)
  data_1.loc[k] = [j,True,False,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),loss]
  print("*"*60)
  print("  ")
  
  print("Minimize b and c")
  b,c,loss = minimize_b_c(X,Y,a,b,c,200000)
  data_1.loc[k+1] = [j,False,True,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),loss]
  print("  ")
  k = k+2
  j = j+1

minimize a
x average at 0 epoch tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000],
        [2.2500, 3.0000]])
yhat at 0 epoch tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
loss at 0 epoch 1.0986123085021973
************************************************************
x average tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000],
        [2.2500, 3.0000]])
Y hat tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
Loss 1.0986123085021973 199999
************************************************************
  
Minimize b and c
x average at 0 epoch tensor([[2.2500, 2.5000],
        [2.2500, 2.5000],
        [2.7500, 2.5000],
        [2.7500, 2.5000],
        [2.2500, 3.0000

In [16]:
# data_1.to_csv("data_1_m_2_2.csv",index=False) 0.005713

In [16]:
data_1

,sno,b_c_fixed,a_fixed,a_value,b_value,c_value,loss
0,0,True,True,"[[0.0], [0.0]]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[0.0, 0.0, 0.0]",1.098612
1,1,True,False,"[[0.0], [0.0]]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[0.0, 0.0, 0.0]",1.098612
2,1,False,True,"[[0.0], [0.0]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.030338
3,2,True,False,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
4,2,False,True,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
5,3,True,False,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
6,3,False,True,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
7,4,True,False,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
8,4,False,True,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999
9,5,True,False,"[[-0.17368719], [-0.24413258]]","[[-1.167538, 14.178669, -13.015318], [-1.55708...","[8.990172, -3.9753044, -5.013558]",0.009999


# m = 9

In [17]:
X1 = torch.tensor([[  [1,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [2,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [1,2],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[1,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[2,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[1,2],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[1,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[2,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[1,2],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[1,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[2,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[1,2],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[2,1],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,2],[3.5,4],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,1],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[2,1],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,2],[3.5,4],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,1],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[2,1],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,2],[3.5,4],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,1],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[2,1],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,2],[3.5,4]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,1]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[2,1]],
                   [  [3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[3.5,4],[1,2]],]) # mosaic data m = 9 , d= 1
Y1 = torch.tensor([0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2])

In [18]:
# softmax(a*X1)
X1.shape,X.shape

(torch.Size([27, 9, 2]), torch.Size([6, 2, 2]))

In [19]:
a = torch.tensor([[1.],[2.]],requires_grad=True)
b = torch.tensor([[0.,0.,0.],[0.,0.,0.]],requires_grad=True)
c = torch.tensor([0.,0.,0.],requires_grad=True)

average = focus_(a,X1)


#print(torch.matmul(X1,a))

# b = 17.38
# c = -32.78
# # #a,loss = minimize_a(X1,Y1,a,b,c)

In [20]:
# a = torch.tensor([0.],requires_grad=True)
# b = torch.tensor([0.],requires_grad=True)
# c = torch.tensor([0.],requires_grad=True)

a = torch.tensor([[0.],[0.]],requires_grad=True)
b = torch.tensor([[0.,0.,0.],[0.,0.,0.]],requires_grad=True)
c = torch.tensor([0.,0.,0.],requires_grad=True)

criterion = CrossEntropyLoss()
data = pd.DataFrame(columns=["sno","b_c_fixed","a_fixed","a_value","b_value","c_value","loss"]) 
#Y1_ = Y1.float()
X1_average = focus_(a,X1)
Yhat1 = classification_(b,c,X1_average)
initial_loss = criterion(Yhat1,Y1)

#print(initial_loss)

k = 0 
data.loc[k] = [k,True,True,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),initial_loss.item()]
k = k+1
j= 1

for i in range(0,40,2):
  print("Minimize b and c")
  b,c,loss = minimize_b_c(X1,Y1,a,b,c,epochs=200000)
  #print(b,c)
  data.loc[k] = [j,False,True,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),loss]
  print("*"*60)
  print("  ")
  print("minimize a")
  

  #print(a,b,c)

  a,loss = minimize_a(X1,Y1,a,b,c,epochs=200000)
  data.loc[k+1] = [j,True,False,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),loss]
  print("  ")
  k = k+2
  j = j+1

Minimize b and c
x average at 0 epoch tensor([[3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778]])
yhat at 0 epoch tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
 

In [21]:
data

,sno,b_c_fixed,a_fixed,a_value,b_value,c_value,loss
0,0,True,True,"[[0.0], [0.0]]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[0.0, 0.0, 0.0]",1.098612
1,1,False,True,"[[0.0], [0.0]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.400275
2,1,True,False,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
3,2,False,True,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
4,2,True,False,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
5,3,False,True,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
6,3,True,False,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
7,4,False,True,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
8,4,True,False,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
9,5,False,True,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999


In [ ]:
# data.to_csv("data_1_m_9_1.csv",index=False)

In [22]:
# a = torch.tensor([0.],requires_grad=True)
# b = torch.tensor([0.],requires_grad=True)
# c = torch.tensor([0.],requires_grad=True)

a = torch.tensor([[0.],[0.]],requires_grad=True)
b = torch.tensor([[0.,0.,0.],[0.,0.,0.]],requires_grad=True)
c = torch.tensor([0.,0.,0.],requires_grad=True)

criterion = CrossEntropyLoss()

data = pd.DataFrame(columns=["sno","b_c_fixed","a_fixed","a_value","b_value","c_value","loss"]) 
#Y1_ = Y1.float()
X1_average = focus_(a,X1)
Yhat1 = classification_(b,c,X1_average)
initial_loss = criterion(Yhat1,Y1)

#print(initial_loss)

k = 0 
data.loc[k] = [k,True,True,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),initial_loss.item()]
k = k+1
j= 1

for i in range(0,40,2):
  
  
  print("minimize a")
  a,loss = minimize_a(X1,Y1,a,b,c,epochs=200000)
  data.loc[k] = [j,True,False,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),loss]

  print("*"*60)
  print("  ")

  print("Minimize b and c")
  b,c,loss = minimize_b_c(X1,Y1,a,b,c,epochs=200000)
  #print(b,c)
  data.loc[k+1] = [j,False,True,a.clone().detach().numpy(),b.clone().detach().numpy(),c.clone().detach().numpy(),loss]
  print("  ")
  k = k+2
  j = j+1

minimize a
x average at 0 epoch tensor([[3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778],
        [3.2222, 3.6667],
        [3.3333, 3.6667],
        [3.2222, 3.7778]])
yhat at 0 epoch tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
       

In [23]:
data

,sno,b_c_fixed,a_fixed,a_value,b_value,c_value,loss
0,0,True,True,"[[0.0], [0.0]]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[0.0, 0.0, 0.0]",1.098612
1,1,True,False,"[[0.0], [0.0]]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[0.0, 0.0, 0.0]",1.098612
2,1,False,True,"[[0.0], [0.0]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.400275
3,2,True,False,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
4,2,False,True,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
5,3,True,False,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
6,3,False,True,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
7,4,True,False,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
8,4,False,True,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999
9,5,True,False,"[[-0.74600106], [-0.8439506]]","[[-1.135506, 18.509396, -17.369743], [-0.43127...","[5.7915044, -2.4555252, -3.337913]",0.009999


In [ ]:
data.to_csv("data_1_m_9_2.csv",index=False)

# m = 50

In [ ]:
X2 = np.ones((50,50))*3
idx = np.arange(0,50,1)
X2[idx,idx] =  -1

X3 = np.ones((50,50))*3
X3[idx,idx] =  1

X3 = np.concatenate((X2,X3),axis=0)
print(X3,X3.shape)

Y3 = np.zeros((100))
Y3[50:] = 1
print(Y3,Y3.shape)

[[-1.  3.  3. ...  3.  3.  3.]
 [ 3. -1.  3. ...  3.  3.  3.]
 [ 3.  3. -1. ...  3.  3.  3.]
 ...
 [ 3.  3.  3. ...  1.  3.  3.]
 [ 3.  3.  3. ...  3.  1.  3.]
 [ 3.  3.  3. ...  3.  3.  1.]] (100, 50)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.] (100,)


In [ ]:
def softmax_(x):
    """Compute softmax values for each sets of scores in x."""
    m =  torch.max(x,dim=1,keepdims=True) 
    print(m)
    e_x = torch.exp(torch.sub(x,m.values)) 
    return e_x / torch.sum(e_x,dim=1,keepdims=True)

In [ ]:
a = torch.tensor([10.],requires_grad=True)
x = torch.tensor([[3.,-1.]])
out = torch.sum(softmax_(a*x) * x,dim=1)
print(out)

torch.return_types.max(
values=tensor([[30.]], grad_fn=<MaxBackward0>),
indices=tensor([[0]]))
tensor([3.], grad_fn=<SumBackward1>)


In [ ]:
out.backward()

In [ ]:
a.grad

tensor([6.7974e-17])

In [ ]:
l = ( torch.exp(a*x[0,0],) + torch.exp(a*x[0,1]) ) **2


#print(l)


f1 = ( ( ( x[0,0] - x[0,1] ) * torch.exp(a*x[0,0] + a*x[0,1]) ) / l  ) * x[0,0]

f2 = ( ( ( x[0,1] - x[0,0] ) * torch.exp(a*x[0,0] + a*x[0,1]) ) / l  ) * x[0,1]
print(f1.item()+f2.item())

6.797367106130223e-17


In [ ]:
x = np.array([[3,-1]])
a = 10
b = 1
c = 0
y = np.array([1])
xhat = np.sum(softmax(a*x) * x,axis=1)
print(xhat)
yhat = classification_(b,c,xhat)

[3.]


In [ ]:
derv_f(x,xhat,y,yhat,a,b)

[6.79736681e-17]


-3.223710561997351e-18